# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
print ('total number of data', len(all_data))

TypeError: a bytes-like object is required, not 'str'

In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [19]:
# YOUR CODE HERE
clf.fit(all_data[training_data], all_target[training_data])

AttributeError: 'function' object has no attribute 'fit'

### predicting

In [10]:
clf.predict(all_data[-1]), all_target[-1]

IndexError: list index out of range

In [11]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [12]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

ValueError: max() arg is an empty sequence

## 4. Evaluate on the test data

In [10]:
expected = []
predicted = []
# YOUR CODE HERE
test_data = permutation[n_training_data:]
expected = all_target[test_data]
predicted = clf.predict(all_data[test_data])

evaluate(expected, predicted)

TypeError: only integer scalar arrays can be converted to a scalar index

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [9]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

TypeError: write() argument must be str, not bytes

Then, in the application we can load the trained classifier again.

In [71]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

(array([10]), 10)